In [2]:
import ExtractorsAndTables as EnT
import Browser as brwsr
import pandas as pd

closed_LID_diamonds = "84559939"
json_data = brwsr.ShippingApi.getShippingAndPaymentInformation(closed_LID_diamonds)

In [3]:
n1 = pd.json_normalize(json_data)

In [4]:
pd.json_normalize(n1["shipping.rates"]).iloc[0]

0    {'region_code': 'ch', 'region_name': 'Switzerl...
1    {'region_code': 'cy', 'region_name': 'Cyprus',...
2    {'region_code': 'gb', 'region_name': 'United K...
3    {'region_code': 'gr', 'region_name': 'Greece',...
4    {'region_code': 'hr', 'region_name': 'Croatia'...
5    {'region_code': 'nl', 'region_name': 'The Neth...
6    {'region_code': 'no', 'region_name': 'Norway',...
7    {'region_code': 'europe', 'region_name': 'Rest...
8    {'region_code': 'dk', 'region_name': 'Denmark'...
9    {'region_code': 'world', 'region_name': 'Anywh...
Name: 0, dtype: object

In [5]:
def json_to_expanded_dataframe(data):
    # Extract the rates data
    rates = data["shipping"]["rates"]

    # Extract other relevant information
    estimated_delivery_times = data["shipping"]["estimated_delivery_times"]
    destination_country = data["shipping"]["destination_country"]
    combined_shipping_allowed = data["shipping"]["combined_shipping_allowed"]
    delivery_methods = data["shipping"]["delivery_methods"]
    extra_insurance = data["shipping"]["extra_insurance"]
    provider_id = data["shipping"]["provider_id"]
    is_pickup_preferable = data["shipping"]["is_pickup_preferable"]
    is_pickup_only = data["shipping"]["is_pickup_only"]
    pickup_location = data["shipping"]["pickup_location"]

    # Add these details to each rate entry
    expanded_rates = []
    for rate in rates:
        rate_entry = rate.copy()
        rate_entry.update({
            "estimated_delivery_from_days": estimated_delivery_times[0]["from_days"],
            "estimated_delivery_to_days": estimated_delivery_times[0]["to_days"],
            "destination_country_name": destination_country["country"]["name"],
            "destination_country_short_code": destination_country["country"]["short_code"],
            "combined_shipping_allowed": combined_shipping_allowed,
            "delivery_methods": ', '.join(delivery_methods),
            "extra_insurance": extra_insurance,
            "provider_id": provider_id,
            "is_pickup_preferable": is_pickup_preferable,
            "is_pickup_only": is_pickup_only,
            "pickup_location_country_code": pickup_location["country_code"],
            "pickup_location_city": pickup_location["city"]
        })
        expanded_rates.append(rate_entry)

    # Create a DataFrame
    df_expanded_rates = pd.DataFrame(expanded_rates)

    return df_expanded_rates

In [6]:
df = json_to_expanded_dataframe(json_data)

In [14]:
df[df["region_name"] == "Denmark"]["price"].iloc[0]

2300.0